### Ollama - Multi Modal Models
In addition to the `ollama_text_generation.ipynb` notebook, this one also supports multi-modal querying - well, in this case you may add an image to your prompt, whereas multi-modal can mean also other media can be used as input.

### Setup
See the repo's tutorial to learn how to spin up the Ollama service

### Available Models
To see, what models are installed on the Raspi, run the cell in the Configuration section of this Notebook

Browse models available for ollama:
https://ollama.com/search

Tested models are:
- `moondream` (1.8b)

--------------------------------------------

#### Discuss Images
Multi-modal models (so far, only `moondream` is tested) can accept more than just text input. In this case we pass a text prompt along with an image to the model. The prompt asks a question about the image.

In [ ]:
# run this cell first
import requests
import lib

url = "http://127.0.0.1:11434/api"
model = "moondream"

stream = True # set False if you want to wait for the final result to be finished
keep_alive = True # set True to improve speed on consecutive prompts to the same model
# but you have to wait 5 minutes until Ollama clears its memory automatically, or manually run:

In [ ]:
# Check Info about Model on Ollama server
lib.check_model(url, model)

In [ ]:
# List locally installed models
lib.list_models(url)

#### without chat history

In [ ]:
data = {
    "model": "moondream",
    "prompt": "what life forms can be seen?",
    "images": [lib.image_file_to_base64("output/dog.png")]
}

# https://github.com/ollama/ollama/issues/7733
result = requests.post(f"{url}/generate", json=data, stream=stream)

response_to_image = lib.extract_response_stream(result) if stream else lib.extract_responses(result)

if not keep_alive:
    lib.unload_model(f"{url}/generate", model)

#### with chat history

In [ ]:
# run this once at the beginning of a chat
messages = []

In [ ]:
# run this for every new chat message you want to send

# put every new prompt in here
prompt = "what is this animal and what does it do?"
image_file = "output/squirrel.png"
last_image = None

# record prompt as message
# only add image if a new image was provided
if image_file != last_image:    
    messages.append(lib.to_message("user", prompt, image=lib.image_file_to_base64(image_file)))
    last_image = image_file
else:
    messages.append(lib.to_message("user", prompt))


data = {
    "model": model,
    "messages": messages,
}

# https://github.com/ollama/ollama/issues/7733
result = requests.post(f"{url}/chat", json=data, stream=stream)

response_to_image = lib.extract_response_stream(result, history=True) if stream else lib.extract_responses(result, history=True)

messages.append(lib.to_message(
    "assistant", 
    lib.without_chain_of_thought(response_to_image, data["model"])
))

if not keep_alive:
    lib.unload_model(f"{url}/generate", data["model"])